In [22]:
EVERYONE_TEXT = 'everyone'

import re
import typing
from collections import defaultdict

In [26]:
class Item:
    def __init__(self, name: str, cost: float, people_to_pay: typing.List) -> None:
        self.name = name
        self.cost = cost
        self.people_to_pay = people_to_pay

    def __repr__(self) -> str:
        return self.__str__()

    def __str__(self) -> str:
        return f'Item {self.name} with cost {self.cost} and buyers {[person.name.capitalize() for person in self.people_to_pay]}'

class Person:
    def __init__(self, name: str, items: typing.List) -> None:
        self.name = name
        self.items = items
    
    def total_cost(self) -> float:
        return sum([item.cost for item in self.items])

    def __repr__(self) -> str:
        return self.__str__()
    
    def __str__(self) -> str:
        return f'{self.name}, in charge of {[item.name.capitalize() for item in self.items]} with a total cost of {self.total_cost()}'

In [27]:
def remove_currency_symbols(cost_with_currency: str) -> str:
    return re.sub(r'[^\d.,]+', '', cost_with_currency)

def get_people_in_line(formatted_people: str, people: typing.List[Person]) -> list:
    people_in_line = [person.lower() for person in re.findall(r'(\w+)', formatted_people)]

    if 'everyone' in people_in_line:
        return people

    extracted_people = [person for person in people if person.name in people_in_line]

    if len(people_in_line) != len(extracted_people):
        people_not_found = [person.name for person in people if person.name not in people_in_line]
        raise BaseException(f'Some people could not be found in the list of names that you provided: {people_not_found}')

    return extracted_people


def get_line_items(line: str, people: typing.List[Person]) -> typing.List[str]:
    pattern = r'(.+): (.+) \((.+)\)'

    matches = re.findall(pattern, line)
    
    if len(matches) == 0:
        raise BaseException(f'Incorrectly formatted line: {pattern}')

    line_items = matches[0]

    if len(line_items) != 3:
        raise BaseException(f'Incorrectly formatted line: {pattern}')
    
    return [
        line_items[0],
        float(remove_currency_symbols(line_items[1])),
        get_people_in_line(line_items[2], people)
    ]

def parse_line(line: str, people: typing.List[Person]) -> Item:
    item, cost, people_to_pay = get_line_items(line, people)
    return Item(name=item, cost=cost, people_to_pay=people_to_pay)

def parse_all_lines(fp: str, people: typing.List[Person]) -> typing.List[Item]:
    with open(fp, 'r') as file:
        lines = file.readlines()
        return [parse_line(line, people) for line in lines]

In [28]:
person_names = input('Please enter all names on this tab, separated by a comma.')

person_names = [name.strip().lower() for name in person_names.split(',')]
people = [Person(name=name, items=[]) for name in person_names]

parse_all_lines('./tabs/costco/costco-tab-08142022.txt', people)

[Item Lemon juice with cost 16.99 and buyers ['Jared'],
 Item Toilet paper with cost 6.33 and buyers ['Jared', 'Carter', 'Taimur'],
 Item Paper towels with cost 6.27 and buyers ['Jared', 'Carter', 'Taimur'],
 Item Peaches with cost 10.99 and buyers ['Carter'],
 Item Rinse aid with cost 3.65 and buyers ['Jared', 'Carter', 'Taimur'],
 Item Dish soap with cost 3.66 and buyers ['Jared', 'Carter', 'Taimur'],
 Item Bounce dryer sheets with cost 2.92 and buyers ['Jared', 'Carter', 'Taimur'],
 Item Magic erasers with cost 3.66 and buyers ['Jared', 'Carter', 'Taimur'],
 Item Steaks with cost 17.34 and buyers ['Jared', 'Carter', 'Taimur'],
 Item Gillette razors with cost 38.99 and buyers ['Jared'],
 Item Degree deodorant with cost 14.99 and buyers ['Jared'],
 Item Cremo shave cream with cost 14.99 and buyers ['Carter'],
 Item Degree deodorant with cost 14.99 and buyers ['Carter'],
 Item Clorox wand with cost 4.99 and buyers ['Jared', 'Carter', 'Taimur'],
 Item Clorox wipes with cost 3.88 and buy